In [1]:
import uproot
import numpy as np
import torch
from torch_geometric.nn import GraphSAGE

import GraphBuilder
import Models 

In [2]:
#########################################################
# Config for our model (I'm annoyed it's here too tbh)
#########################################################
N_NODE_FEATURES = 23
HIDDEN_CHANNELS = 4
NUM_LAYERS = 2
USE_GRAPHSAGE = True

GNN_MODEL_PATH = "/Users/isobel/Desktop/DUNE/2024/Hierarchy/models/gnn_model_primaries_oneway.pt"
CLASSIFICATION_MODEL_PATH = "/Users/isobel/Desktop/DUNE/2024/Hierarchy/models/classifier_model_primaries_oneway.pt"

PRIMARY_THRESHOLD = 0.3

############################
# Setup the model for eval
############################

if (USE_GRAPHSAGE) :
    gnn_model = GraphSAGE(N_NODE_FEATURES, hidden_channels=HIDDEN_CHANNELS, num_layers=NUM_LAYERS)
else :
    gnn_model = Models.GNN(N_NODE_FEATURES, HIDDEN_CHANNELS)

classifier_model = Models.Classifier()


gnn_model.load_state_dict(torch.load(GNN_MODEL_PATH))
gnn_model.eval()

classifier_model.load_state_dict(torch.load(CLASSIFICATION_MODEL_PATH))
classifier_model.eval()

RuntimeError: Error(s) in loading state_dict for GraphSAGE:
	size mismatch for convs.0.lin_l.weight: copying a param with shape torch.Size([16, 23]) from checkpoint, the shape in current model is torch.Size([4, 23]).
	size mismatch for convs.0.lin_l.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for convs.0.lin_r.weight: copying a param with shape torch.Size([16, 23]) from checkpoint, the shape in current model is torch.Size([4, 23]).
	size mismatch for convs.1.lin_l.weight: copying a param with shape torch.Size([16, 16]) from checkpoint, the shape in current model is torch.Size([4, 4]).
	size mismatch for convs.1.lin_l.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for convs.1.lin_r.weight: copying a param with shape torch.Size([16, 16]) from checkpoint, the shape in current model is torch.Size([4, 4]).

In [ ]:
#############################
# Let's read the file
#############################

fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/nue_dune10kt_1x2x6_1124_526_20230828T185125Z_gen_g4_detsim_hitreco_ccnutree'#'/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/ccnutree_higherStats'
#fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/ccnutree_higherStats'
inputFileName = fileName + '.root'
#outputFileSuffix = '_ParticleToParticleTruth.pt'
#outputFileSuffix = '_NeutrinoToParticleTruth.pt'

treeFile = uproot.open(inputFileName)
tree = treeFile['ccnuselection/ccnusel']
branches = tree.arrays()

In [ ]:
#############################
# Get event-level stuff
#############################
run = np.array(branches['Run'])
subrun = np.array(branches['SubRun'])
event = np.array(branches['Event'])

recoNuVertexX = branches['RecoNuVtxX']
recoNuVertexY = branches['RecoNuVtxY']
recoNuVertexZ = branches['RecoNuVtxZ']
    
#############################
# Get pfp-level stuff - these cannot be numpy arrays...
#############################
trackShowerScore_main = branches['RecoPFPTrackShowerScore']
nHits_main = branches['RecoPFPRecoNHits']
charge_main = branches['RecoPFPRecoCharge']
vertexX_main = branches['RecoPFPRecoVertexX']
vertexY_main = branches['RecoPFPRecoVertexY']
vertexZ_main = branches['RecoPFPRecoVertexZ']
trackEndX_main = branches['RecoTrackRecoEndX']
trackEndY_main = branches['RecoTrackRecoEndY']
trackEndZ_main = branches['RecoTrackRecoEndZ']
showerDirX_main = branches['RecoShowerRecoDirX']  # not the best direction estimate, placeholder
showerDirY_main = branches['RecoShowerRecoDirY']
showerDirZ_main = branches['RecoShowerRecoDirZ']
ivysaurusMuon_main = branches['RecoPFPIvysaurusMuon']
ivysaurusProton_main = branches['RecoPFPIvysaurusProton']
ivysaurusPion_main = branches['RecoPFPIvysaurusPion']
ivysaurusElectron_main = branches['RecoPFPIvysaurusElectron']
ivysaurusPhoton_main = branches['RecoPFPIvysaurusPhoton']
wiggliness_main = branches['RecoTrackDeflecAngleSD']
trackLength_main = branches['RecoTrackLength']
displacement_main = branches['RecoShowerPandrizzleDisplacement']
dca_main = branches['RecoShowerPandrizzleDCA']
recoGeneration_main = branches['RecoPFPRecoGeneration']
recoSelf_main = branches['RecoPFPSelf']
pathwayLength_main = branches['RecoShowerPandrizzlePathwayLengthMin']
pathwayScatteringAngle2D_main = branches['RecoShowerPandrizzleMaxShowerStartPathwayScatteringAngle2D']
nShowerStartHits_main = branches['RecoShowerPandrizzleMaxNPostShowerStartHits']
showerScatterAngle_main = branches['RecoShowerPandrizzleMaxPostShowerStartScatterAngle']
nuVertexEnergyAsymmetry_main = branches['RecoShowerPandrizzleMaxPostShowerStartNuVertexEnergyAsymmetry']
showerStartEnergyAsymmetry_main = branches['RecoShowerPandrizzleMaxPostShowerStartShowerStartEnergyAsymmetry']
nuVertexEnergyWeightedMeanRadialDistance_main = branches['RecoShowerPandrizzleMaxPostShowerStartNuVertexEnergyWeightedMeanRadialDistance']
showerMoliereRadius_main = branches['RecoShowerPandrizzleMinPostShowerStartShowerStartMoliereRadius']
showerOpeningAngle_main = branches['RecoShowerPandrizzleMaxPostShowerStartOpeningAngle']
foundHitRatio_main = branches['RecoShowerPandrizzleMaxFoundHitRatio']
initialGapSize_main = branches['RecoShowerPandrizzleMaxInitialGapSize']
largestProjectedGapSize_main = branches['RecoShowerPandrizzleMinLargestProjectedGapSize']
nViewsWithAmbiguousHits_main = branches['RecoShowerPandrizzleNViewsWithAmbiguousHits']
ambiguousHitMaxUnaccountedEnergy_main = branches['RecoShowerPandrizzleAmbiguousHitMaxUnaccountedEnergy']

# True information.. (for cheating)
pfpTrueMomX_main = branches['RecoPFPTrueMomX']
pfpTrueMomY_main = branches['RecoPFPTrueMomY']
pfpTrueMomZ_main = branches['RecoPFPTrueMomZ']
pfpTruePDG_main = branches['RecoPFPTruePDG']

#############################
# Network truth - these cannot be numpy arrays...
#############################
trueVisibleGeneration_main = branches['RecoPFPTrueVisibleGeneration']
trueTrackID_main = branches['RecoPFPTrueTrackID']
trueVisibleParentTrackID_main = branches['RecoPFPTrueVisibleParentTrackID']

#############################
# How many entries are we working with?
#############################
nEntries = run.shape[0]
print('We are working with:', nEntries, 'entries')

#############################
# Rebuilt Hierarchy
#############################
newParentTrackID_main = []
newParentGen_main = []

In [ ]:
#############################
# Find Primaries - Let's create our graphs!
#############################

iEvent = 0

modeDict = {
    "ADD_NEUTRINO"                 : True,
    "CHEAT_DIRECTION"              : True,
    "CHEAT_PID"                    : True, 
    "MAKE_PARTICLE_PARTICLE_LINKS" : True,
    "EDGE_FRACTION"                : 0.8,
    "DO_NORMALISATION"             : True,
    "IS_PRIMARY_TRAINING"          : False,
    "IS_HIGHER_TIER_TRAINING"      : False
}

for iEvent in range(nEntries) : 
        
    eventDict = {
        "recoNuVertexX"                            : recoNuVertexX[iEvent], \
        "recoNuVertexY"                            : recoNuVertexY[iEvent], \
        "recoNuVertexZ"                            : recoNuVertexZ[iEvent], \
        "nParticles"                               : np.array(pfpTruePDG_main[iEvent]).shape[0], \
        "trackShowerScore"                         : np.array(trackShowerScore_main[iEvent], dtype='f'), \
        "nHits"                                    : np.array(nHits_main[iEvent], dtype='f'), \
        "charge"                                   : np.array(charge_main[iEvent], dtype='f'), \
        "vertexX"                                  : np.array(vertexX_main[iEvent], dtype='f'), \
        "vertexY"                                  : np.array(vertexY_main[iEvent], dtype='f'), \
        "vertexZ"                                  : np.array(vertexZ_main[iEvent], dtype='f'), \
        "trackEndX"                                : np.array(trackEndX_main[iEvent], dtype='f'), \
        "trackEndY"                                : np.array(trackEndY_main[iEvent], dtype='f'), \
        "trackEndZ"                                : np.array(trackEndZ_main[iEvent], dtype='f'), \
        "showerDirX"                               : np.array(showerDirX_main[iEvent], dtype='f'), \
        "showerDirY"                               : np.array(showerDirY_main[iEvent], dtype='f'), \
        "showerDirZ"                               : np.array(showerDirZ_main[iEvent], dtype='f'), \
        "ivysaurusMuon"                            : np.array(ivysaurusMuon_main[iEvent], dtype='f'), \
        "ivysaurusProton"                          : np.array(ivysaurusProton_main[iEvent], dtype='f'), \
        "ivysaurusPion"                            : np.array(ivysaurusPion_main[iEvent], dtype='f'), \
        "ivysaurusElectron"                        : np.array(ivysaurusElectron_main[iEvent], dtype='f'), \
        "ivysaurusPhoton"                          : np.array(ivysaurusPhoton_main[iEvent], dtype='f'), \
        "trackLength"                              : np.array(trackLength_main[iEvent], dtype='f'), \
        "displacement"                             : np.array(displacement_main[iEvent], dtype='f'), \
        "dca"                                      : np.array(dca_main[iEvent], dtype='f'), \
        "isNeutrinoPDG"                            : np.zeros(np.array(pfpTruePDG_main[iEvent]).shape), \
        "nuVertexEnergyAsymmetry"                  : np.array(nuVertexEnergyAsymmetry_main[iEvent], dtype='f'), \
        "nuVertexEnergyWeightedMeanRadialDistance" : np.array(nuVertexEnergyWeightedMeanRadialDistance_main[iEvent], dtype='f'), \
        "trueTrackID"                              : np.array(trueTrackID_main[iEvent]), \
        "trueVisibleParentTrackID"                 : np.array(trueVisibleParentTrackID_main[iEvent]), \
        "trueMomX"                                 : np.array(pfpTrueMomX_main[iEvent]), \
        "trueMomY"                                 : np.array(pfpTrueMomY_main[iEvent]), \
        "trueMomZ"                                 : np.array(pfpTrueMomZ_main[iEvent]), \
        "truePDG"                                  : np.array(pfpTruePDG_main[iEvent])
    }

    data_pos, data_neg, data_FC, pfp_index = GraphBuilder.GraphBuilder(eventDict, modeDict)
    # If these are empty, we're going to have to do something about that...
    
    ##################################################
    # Calculate Scores of neutrino -> particle edges
    ##################################################
    pred = gnn_model(data_FC.x, data_FC.edge_index)

    # Assuming that the neutrino is the last node added (which it is in this config)
    nuEdgeMask = (data_FC.edge_index[0] == (data_FC.num_nodes - 1))

    target_index = data_FC.edge_index[0][nuEdgeMask].detach().numpy()
    source_index = data_FC.edge_index[1][nuEdgeMask].detach().numpy()
    edge_index = torch.tensor([target_index, source_index], dtype=torch.long)

    edge_pred = classifier_model(pred, edge_index)
    
    ############################
    # Find Primaries
    ############################
    newParentSelf = [-1] * eventDict["nParticles"]
    newParentGen = [-1] * eventDict["nParticles"]
    newParentSelf[-1] = -999 # Nu has no parent
    newParentGen[-1] = 1 

    particle_tiers = [[]]

    for iParticle in range(eventDict["nParticles"]) :
    
        # Move on if it is the neutrino
        if (eventDict['isNeutrinoPDG'][iParticle] == 1) :
            continue
    
        primary_GNN_score = edge_pred[np.where(pfp_index == iParticle)[0][0]].item()
    
        if (primary_GNN_score > PRIMARY_THRESHOLD) :
            newParentSelf[iParticle] = eventDict["nParticles"] # nu has self == nParticles
            newParentGen[iParticle] = 2
            particle_tiers[0].append(iParticle)
    
    ############################
    # Find Higher Tiers
    ############################    
    while (len(particle_tiers[-1]) != 0) :
        
        tier = []
        
        for iChild in range(eventDict["nParticles"]) :
            
            if (eventDict["isNeutrinoPDG"][iChild] == 1) :
                continue
            
            # Has it been assigned yet?
            if (newParentGen[iChild] != -1) :
                continue
        
            for iParent in particle_tiers[-1]:
                
                # I dont think that this can happen
                if (eventDict["isNeutrinoPDG"][iParent] == 1) :
                    continue
                
                if (eventDict["trueTrackID"][iParent] == eventDict["trueVisibleParentTrackID"][iChild]) :
                    newParentSelf[iChild] = recoSelf_main[iEvent][iParent]
                    newParentGen[iChild] = len(particle_tiers) + 2
                    tier.append(iChild)
                    
        particle_tiers.append(tier)
        
    ######################################
    # Say that any remaining are primaries
    ###################################### 
    for iParticle in range(eventDict["nParticles"]) :
    
        # Move on if it is the neutrino
        if (eventDict['isNeutrinoPDG'][iParticle] == 1) :
            continue
            
        if (newParentGen[iParticle] == -1) :
            newParentSelf[iParticle] = eventDict["nParticles"] # nu has self == nParticles
            newParentGen[iParticle] = 2
            particle_tiers[0].append(iParticle)    
        
    print('------------------------')
    print('newParentTrackID:     ', newParentSelf)
    print('newParentGen:         ', newParentGen)
    print('trueVisibleGeneration:', trueVisibleGeneration_main[iEvent])
    print('particle_tiers:', particle_tiers)
    print('------------------------')
                
    newParentTrackID_main.append(newParentSelf)
    newParentGen_main.append(newParentGen)    
 